In [1]:
from tensorflow.python.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
import numpy as np
from numpy import argmax
import os
import cv2 #이미지 읽기용
import matplotlib.pyplot as plt
from numpy import exp #Softmax출력용 

In [2]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
list = [[0]*4]*100 # 1 x 4 배열을 100개 만듬

In [3]:
img = cv2.imread('face.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3,5)
count = 0 #반복 실행 시 덮어쓰도록 여기에 선언

for (x,y,w,h) in faces: #좌표 값과 rectangular의 width height를 받게 된다.
    # x,y값은 rectangular가 시작하는 지점의 좌표
    # x, y값은 좌표값 w,h는 얼굴 높이 너비
    # 원본 이미지에 얼굴의 위치를 표시하는 작업을 함.
    # for문을 돌리는 이유는 여러 개가 검출 될 수 있기 때문.

    list[count][0] = x
    list[count][1] = y
    list[count][2] = w
    list[count][3] = h
    print(list[count])
    #image 좌표값대로 잘라서 따로 저장하는 부분
    img_trim = img[y:y + h,x:x + w]  # trim한 결과를 img_trim에 담는다
    cv2.imwrite('target_{}.jpg'.format(count),img_trim) #각 카운터에 맞는 이름으로 저장
    
    count = count + 1

    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

[57, 96, 199, 199]
[260, 64, 190, 190]


In [4]:
img.shape

(330, 540, 3)

In [5]:
cv2.imshow('image',img)
cv2.waitKey()

-1

In [5]:
#분석 준비
image_size=48
labels=['neutral', 'fear', 'happy', 'sad', 'angry', 'surprise']
analy_image=[]
img_count = 0

#데이터 전처리
for index in faces:
    image = cv2.imread("target_{}.jpg".format(img_count)) #이미지 읽기
    image = cv2.resize(image, (image_size,image_size)) #이미지 사이즈 편집
    image = img_to_array(image) #이미지 배열화
    print(image.shape)
    analy_image.append(image)
    img_count += 1

(48, 48, 3)
(48, 48, 3)


In [ ]:
analy_image = np.array(analy_image, dtype="float32") / 255.0
labels = np.array(labels)
mlb = LabelBinarizer()
labels = mlb.fit_transform(labels)

In [6]:
model = load_model('VGG16_complete.h5')

In [8]:
image_predict = model.predict(analy_image) #사진 분석

ValueError: in user code:

    C:\Users\Otakuking\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\Otakuking\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Otakuking\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Otakuking\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Otakuking\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Otakuking\anaconda3\lib\site-packages\tensorflow\python\keras\engine\tr

In [ ]:
#결과 출력
label = labels[image_predict[0].argmax()]
confidence = image_predict[0][image_predict[0].argmax()]
print('{} {:.2f}%'.format(label, confidence * 100)) 

In [ ]:
def softmax(vector):
    e = exp(vector)
    return e / e.sum()
 
# define data
data = [1, 3, 2]
# convert list of numbers to a list of probabilities
result = softmax(data)
# report the probabilities
print(result)

In [ ]:
# softmax function
def softmax_func(self, x_data):
    predictions = x_data - (x_data.max(axis=1).reshape([-1, 1]))
    softmax = np.exp(predictions)
    softmax /= softmax.sum(axis=1).reshape([-1, 1])
    return softmax